# Tritium desorption in fusion reactor walls

In this example, we look at the ability of `twinLab` to model the [desorption](https://en.wikipedia.org/wiki/Desorption) (the physical process where a previously adsorbed substance is released from a surface) of tritium (a radioactive isotope of hydrogen) in the wall of a nuclear fusion reactor. Fusion generates almost no radioactive waste, and the little waste that it does produce has a short half-life. However, the interior of the reactor wall is bombarded by a high-neutron flux during fusion, far higher than any naturally-occurring radioactive process, and therefore the properties of materials under high neutron bombardment are unknown. Computer simulations are required to understand the material properties in such extreme circumstances, but simulations are expensive in terms of computational power, and cannot be run at every point in parameter space under consideration. `twinLab` can be used to train simulation surrogate models using data from a sparse array of simulations. This allows for meaningful interpolation and extrapolation to unexplored regions of parameter space, together with a calibrated uncertainty estimate on the accuracy of the simulation surrogate.

First, import the required libraries

In [1]:
# Standard imports
import os
from pprint import pprint

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Project imports
import twinlab as tl


         === TwinLab Client Initialisation ===
         Version  : 1.3.1
         User     : alexander
         Server   : https://qisuqloa39.execute-api.eu-west-2.amazonaws.com/prod
         Key      : dg4Je8SwoFZ-o_KdVOpvkg



Now, define some parameters. We need to provide `twinLab` with locations of data files (in `.csv` format) and directories, together with a chosen surrogate campaign name (here `ukaea`) and a list of the input parameters that we are going to use as inputs to our model. Here these 5 parameters are $E_1$, $E_2$, $E_3$, $n_1$, and $n_2$ and quantify properties of the material that can trap tritium nuclei. The $E_i$ parameters are energies while the $n_i$ parameters are particle number densities. $E_1$ and $n_1$ pertain to the same trap, as do $E_2$ and $n_2$, but in the model parameterized by $E_3$ the value of $n_3$ is calculated internally, and evolves with time, and therefore this is not an input parameter for our training regime.

In [2]:
# Files and file paths
file_train = os.path.join("resources", "datasets", "ukaea.csv")
campaign_dir = os.path.join("resources", "campaigns", "ukaea")
file_grid = os.path.join(campaign_dir, "grid.csv")
# file_eval = campaign_dir + "eval.csv"
# file_eval = campaign_dir + "post.csv"
file_eval = os.path.join(campaign_dir, "test.csv")

# Parameters
dataset_id = "fusion"
campaign_id = dataset_id
inputs = ["E1", "E2", "E3", "n1", "n2"]

Load the necessary data from `.csv` files for training and plotting

In [3]:
df_train = pd.read_csv(file_train)
df_eval = pd.read_csv(file_eval)
df_grid = pd.read_csv(file_grid, header=None)
display(df_train)

,E1,E2,E3,n1,n2,y0,y1,y2,y3,y4,...,y614,y615,y616,y617,y618,y619,y620,y621,y622,y623
0,0.943244,0.999344,1.496880,0.000609,0.000431,8.767405e+14,9.683132e+14,1.059957e+15,1.165661e+15,1.283038e+15,...,4.369955e+16,4.317858e+16,4.276888e+16,4.247459e+16,4.218029e+16,4.188600e+16,4.159170e+16,4.129741e+16,4.100311e+16,4.070882e+16
1,0.806345,1.120079,1.169154,0.003404,0.000660,9.210712e+16,9.995254e+16,1.084585e+17,1.176555e+17,1.275583e+17,...,1.128383e+17,1.112052e+17,1.095722e+17,1.097652e+17,1.100055e+17,1.102458e+17,1.104861e+17,1.107265e+17,1.109668e+17,1.112071e+17
2,0.729418,1.071554,1.604596,0.002666,0.000797,1.685389e+17,1.814004e+17,1.952128e+17,2.100128e+17,2.258109e+17,...,9.592846e+16,9.391916e+16,9.190986e+16,9.092210e+16,9.032004e+16,8.971799e+16,8.911593e+16,8.851388e+16,8.791183e+16,8.730977e+16
3,0.864302,1.247695,1.267029,0.004300,0.000125,3.530973e+16,3.855547e+16,4.210757e+16,4.597684e+16,5.017253e+16,...,1.498126e+17,1.476732e+17,1.455338e+17,1.437080e+17,1.432555e+17,1.428030e+17,1.423504e+17,1.418979e+17,1.414454e+17,1.409929e+17
4,0.893397,1.012177,1.190578,0.004729,0.000265,1.770521e+16,1.939110e+16,2.124748e+16,2.327789e+16,2.548775e+16,...,5.430334e+16,5.392261e+16,5.354189e+16,5.316116e+16,5.278044e+16,5.239971e+16,5.201899e+16,5.163826e+16,5.125754e+16,5.087681e+16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,0.893503,1.130860,1.354340,0.000518,0.000561,2.765631e+15,3.015804e+15,3.299841e+15,3.613701e+15,3.955823e+15,...,1.216010e+17,1.205185e+17,1.194360e+17,1.183536e+17,1.172711e+17,1.161886e+17,1.151061e+17,1.140237e+17,1.129412e+17,1.118587e+17
4092,0.864224,0.916487,1.116019,0.001233,0.000702,1.278325e+16,1.395800e+16,1.525427e+16,1.666896e+16,1.820456e+16,...,2.787106e+16,2.756250e+16,2.725395e+16,2.694540e+16,2.663685e+16,2.638756e+16,2.637311e+16,2.635866e+16,2.634420e+16,2.632975e+16
4093,0.729359,1.190237,1.428134,0.002938,0.000474,1.818643e+17,1.957515e+17,2.106642e+17,2.266445e+17,2.437041e+17,...,1.832247e+17,1.814117e+17,1.795988e+17,1.777858e+17,1.759729e+17,1.741599e+17,1.723470e+17,1.705340e+17,1.687211e+17,1.669081e+17
4094,0.806240,1.038786,1.335457,0.001989,0.000280,6.772855e+16,7.349775e+16,7.976372e+16,8.654131e+16,9.384140e+16,...,4.172277e+16,4.131412e+16,4.090546e+16,4.049681e+16,4.008815e+16,3.967950e+16,3.927084e+16,3.886219e+16,3.845353e+16,3.804488e+16


Next, we create the parameter dictionary that we need to give to run twinLab.  At a minimum the user must provide the `filename` of the dataset on which we want to train our model (`.csv` format), together with the columns that we will take to be the `inputs` and `outputs` of our model, once that is trained. By default, `twinLab` will use all of the data for training. This latter choice can be overridden by adding e.g., `train_test_split=1000` so that only the first 1000 entries in `filename` are used for training, and the remaining examples are then used for internal model evaluation.

In this case we are training a functional model, which means that we want to return a *function* at every point in parameter space ($E_1$, $E_2$, $E_3$, $n_1$, $n_2$). In this case, our function describes the tritium desorption rate, $D$, of the material of the reactor wall as a function of temperature (rate of emitted nuclei per reactor wall area). The training of the surrogate is agnostic to the values of reactor temperature, $T$, so we must provide this by hand (`df_grid` above, from `file_grid`). The output of our model will therefore be the function $D(T; E_1, E_2, E_3, n_1, n_2)$.

The `twinLab` model achieves this by predicting the value of $D$ at $\sim 500$ points in $T$ in a regularly-spaced grid between $300\mathrm{K}$ and $800\mathrm{K}$. The correlations between points adjacent in $T$ are incorporated naturally by the model, and `twinLab` provides a model uncertainty. Here we call the outputs `y`, rather than `D`, as per the typical data-science convention.

In [4]:
# Column headings for outputs
y_outputs = [f"y{i}" for i in range(len(df_grid))]

# Parameters
params = {
    "dataset_id": dataset_id,
    "inputs": inputs,
    "outputs": y_outputs,
    "decompose_outputs": True,
    "train_test_ratio": 0.25,
}

Now the dataset can be uploaded to the cloud

In [5]:
tl.upload_dataset(file_train, dataset_id, verbose=True)

File resources/datasets/ukaea.csv is uploading.
Processing dataset.
Dataset fusion was processed.


List all of the datasets under our username to check that the upload worked

In [6]:
_ = tl.list_datasets(verbose=True)

Datasets:
['data-error', 'data-error-std', 'fusion', 'full-matrix-stdv']


Print some useful properties of the dataset

In [7]:
_ = tl.query_dataset(dataset_id, verbose=True)

Dataset summary:
                E1           E2           E3           n1           n2  \
count  4096.000000  4096.000000  4096.000000  4096.000000  4096.000000   
mean      0.850000     1.100000     1.425000     0.002750     0.000550   
std       0.086613     0.115484     0.187662     0.001299     0.000260   
min       0.700046     0.900088     1.100129     0.000500     0.000100   
25%       0.775034     1.000053     1.262581     0.001626     0.000325   
50%       0.849975     1.099983     1.425016     0.002750     0.000550   
75%       0.924985     1.199979     1.587442     0.003875     0.000775   
max       0.999995     1.299965     1.749911     0.004999     0.001000   

                 y0            y1            y2            y3            y4  \
count  4.096000e+03  4.096000e+03  4.096000e+03  4.096000e+03  4.096000e+03   
mean   6.090767e+16  6.581552e+16  7.111414e+16  7.682121e+16  8.294281e+16   
std    6.766648e+16  7.287118e+16  7.845608e+16  8.444440e+16  9.084136e+16   


Train the surrogate model, this step takes around 6 minutes.

In [8]:
tl.train_campaign(params, campaign_id, verbose=True)

Model fusion has begun training.


List the campaigns again to ensure that training has been completed

In [ ]:
_ = tl.list_campaigns(verbose=True)

Query the campaign to check how training has proceeded

In [ ]:
_ = tl.query_campaign(campaign_id, verbose=True)

Evaluate the trained emulator on `X` ($E_1, E_2, E_3, n_1, n_2$) from the evaluation file

In [ ]:
df_mean, df_std = tl.predict_campaign(file_eval, campaign_id, verbose=False)
display(df_mean)
display(df_std)

Plot the results for a few different `y` values to sanity check. These values correspond to the tritium desorption flux at different reactor temperatures: $D(T_i)$.

In [ ]:
# Parameters for plot
color = "blue"
alpha = 0.8
xs = {"E1": r"$E_{1}$", "E2": r"$E_{2}$", "E3": r"$E_{3}$", "n1": r"$n_{1}$", "n2": r"$n_{2}$"}
ys = {f"y{i}": fr"$y_{{{i}}}$" for i in [0, 100, 150]}

# Plot some examples
nrow, ncol = len(ys), len(xs)
plt.subplots(nrow, ncol, figsize=(25, 10))
nplot = 0
for y, y_label in ys.items():
    for x, x_label in xs.items():
        nplot += 1
        plt.subplot(nrow, ncol, nplot)
        plt.errorbar(df_eval[x], df_mean[y], yerr=df_std[y], marker='.', lw=1, ls='None', color=color, alpha=alpha, label="Model")
        plt.plot(df_train[x], df_train[y], ".", color="black", alpha=0.1, label="Training data")
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        if nplot==1: plt.legend()
plt.show()

We are now in a position to plot the output of the trained surrogate model together with the "truth" from the evaluation file. The intensity of the blue regions correspond to the probability as predicted by the surrogate. It can be seen that the truth mainly goes through the regions of high probability as predicted by the model. Thus we have trained an accurate surrogate for the tritium desorption process. This training was done on the cloud and the surrogate has been stored and can be evaluated extremely quickly, with uncertainty predictions built into the modelling.

In [ ]:
# Parameters for plot
error_inflation_factor = 1. # Factor to multiply error by for plotting
y_fac = 18 # Factor to divide y by for plotting [log10]
plot_eval = True
data_alpha = 0.75
plot_model_mean = True
plot_model_bands = True
plot_model_blur = False
nsigs = [1, 2]
model_alpha = 0.5
n_model_blur = 100
model_color = 'blue'
number_of_training_examples = 0
number_of_model_examples = 5

# Plot results
grid = df_grid.iloc[:, 0]
plt.subplots()
if (plot_model_blur or plot_model_bands) and not plot_model_mean: 
    plt.fill_between(grid, np.nan, np.nan, color=model_color, alpha=model_alpha, lw=0., label="Model predictions")
for example in range(number_of_training_examples): # Training examples
    train = df_train[y_outputs].iloc[example]/10**y_fac
    label = "Example training data" if example==0 else None
    plt.plot(grid, train, color='black', alpha=0.5, label=label)
for example in range(number_of_model_examples): # Model predictions
    mean = df_mean[y_outputs].iloc[example]/10**y_fac
    err = error_inflation_factor*df_std[y_outputs].iloc[example]/10**y_fac
    if plot_eval and (file_eval == campaign_dir + "test.csv"):
        eval = df_eval[y_outputs].iloc[example]/10**y_fac
        label = "Test data" if example==0 else None
        plt.plot(grid, eval, color='black', alpha=data_alpha, label=label)
    if plot_model_bands:
        for isig, nsig in enumerate(nsigs):
            label = "Model predictions" if (isig == 0) and (example == 0) else None
            plt.fill_between(grid, mean-nsig*err, mean+nsig*err, color=model_color, alpha=model_alpha/(isig+1), lw=0., label=label)
    if plot_model_blur and not plot_model_bands:
        alpha = tl.get_blur_alpha(n_model_blur, model_alpha)
        dys = tl.get_blur_boundaries(n_model_blur)
        for iy, dy in enumerate(dys):
            label = "Model predictions" if (iy == 0) and (example == 0) else None
            plt.fill_between(grid, mean-dy*err, mean+dy*err, color=model_color, alpha=alpha, lw=0., label=label)
    if plot_model_mean:
        label = "Model predictions" if (example==0) and (not plot_model_bands) and (not plot_model_blur) else None
        plt.plot(grid, mean, color=model_color, label=label, alpha=model_alpha)
plt.xlabel(r'Temperature [K]')
plt.xlim((grid.min(), grid.max()))
plt.ylabel(rf"Desorption rate [$10^{{{y_fac}}}$ $m^{{{-2}}}$ $s^{{{-1}}}$]")
plt.ylim(bottom=0.)
plt.legend()
plt.show()

Finally, delete the campaign if necessary...

In [ ]:
tl.delete_campaign(campaign_id, verbose=True)

... and delete the dataset if necessary.

In [ ]:
tl.delete_dataset(dataset_id, verbose=True)